In [4]:
from glob import glob
# import sys
from dotenv import load_dotenv

load_dotenv()
# sys.path.append("/u/zliu/datastor1/KE-by-CP")
import pandas as pd
# from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
from collections import Counter
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import describe
from thefuzz import fuzz

from datasets import load_dataset, load_from_disk

from copy import deepcopy

from dateutil.parser import parse
from dateutil.parser import ParserError

from collections import defaultdict

import re

def is_date(string):
    try:
        parse(string)
        return True
    except ParserError:
        return False

In [ ]:
common_fact_date_dataset = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/common_date_data/common_date_question_generation.hf")

In [ ]:
common_fact_date_dataset

Dataset({
    features: ['topic', 'qa_pairs'],
    num_rows: 31
})

In [14]:
unique_questions = set()
common_fact_date_df_content = []
for i in range(len(common_fact_date_dataset)):
    topic_facts = common_fact_date_dataset[i]
    topic = topic_facts["topic"]
    for qa in topic_facts["qa_pairs"]:
        
        if qa["question"] in unique_questions:
            continue
        unique_questions.add(qa["question"])
        qa["topic"] = topic
        common_fact_date_df_content.append(qa)
        

In [ ]:
common_fact_date_df = pd.DataFrame(common_fact_date_df_content)
common_fact_date_df.to_excel(f"{vars.DATA_DIR}/debug_meta_train/common_date_data/common_fact_date.xlsx", index=False)

In [ ]:
common_fact_date_rewrite_df = pd.read_excel(f"{vars.DATA_DIR}/debug_meta_train/common_date_data/common_fact_date_w-rewrite.xlsx")

In [60]:
# increment answer by 1 year
common_fact_date_rewrite_increment_df_content = []
bc_year_count = 0
s_year_count = 0
interval_year_count = 0
for r in common_fact_date_rewrite_df.to_dict("records"):
    new_r = deepcopy(r)
    
    if "bc" in r["answer"].lower():
        bc_year_count += 1
        continue
    if "s" in r["answer"].lower() or "c" in r["answer"].lower():
        s_year_count += 1
        continue
    if "-" in r["answer"].lower():
        interval_year_count += 1
        continue
    assert is_date(r["answer"]), r
    
    assert r["answer"].isdigit(), r
    new_r["rewrite_answer"] = str(int(r["answer"]) + 1)
    new_r["original_answer"] = new_r["answer"]
    new_r["original_question"] = new_r["question"]
    new_r["answer"] = new_r["rewrite_answer"]
    new_r["question"] = new_r["rewrite_question"]
    del new_r["rewrite_answer"]
    del new_r["rewrite_question"]
    common_fact_date_rewrite_increment_df_content.append(new_r)
len(common_fact_date_rewrite_increment_df_content)

1411

In [ ]:
common_fact_date_rewrite_increment_df = pd.DataFrame(common_fact_date_rewrite_increment_df_content)
common_fact_date_rewrite_increment_df.to_excel(f"{vars.DATA_DIR}/debug_meta_train/common_date_data/common_fact_date_w-rewrite_increment.xlsx", index=False)

io.dump_jsonlines(common_fact_date_rewrite_increment_df_content, f"{vars.DATA_DIR}/debug_meta_train/common_date_data/common_fact_date_w-rewrite_increment.jsonl")

In [ ]:
rand_shuffle = np.arange(len(common_fact_date_rewrite_increment_df_content))
np.random.shuffle(rand_shuffle)

n_dev = 100
n_train = len(common_fact_date_rewrite_increment_df_content) - n_dev
train = [common_fact_date_rewrite_increment_df_content[i] for i in rand_shuffle[:n_train]]
valid = [common_fact_date_rewrite_increment_df_content[i] for i in rand_shuffle[n_train:]]
io.dump_jsonlines(train, f"{vars.DATA_DIR}/debug_meta_train/common_date_data/train.jsonl")

io.dump_jsonlines(valid, f"{vars.DATA_DIR}/debug_meta_train/common_date_data/valid.jsonl")

In [63]:
train[0]

{'answer': '1882',
 'question': 'When was the year after the year that the start of the Scramble for Africa happened?',
 'topic': 'Colonization and Empire Building',
 'original_answer': '1881',
 'original_question': 'What year marks the start of the Scramble for Africa?'}

In [64]:
common_fact_date_rewrite_increment_df_content[0]

{'answer': '1733',
 'question': 'When was the year after the year that George Washington was born?',
 'topic': "Historic Figures' Birthday",
 'original_answer': '1732',
 'original_question': 'In which year was George Washington born?'}

In [ ]:
common_date_year_after = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/common_date_data_year_after/valid.jsonl")

In [12]:
common_date_year = []

for d in common_date_year_after:
    new_d = {
        "question": d["original_question"],
        "answer": d["original_answer"],
        "topic": d["topic"],
        "year_after_question": d["question"],
        "year_after_answer": d["answer"],
    }
    common_date_year.append(new_d)
len(common_date_year)

100

In [ ]:
io.dump_jsonlines(common_date_year, f"{vars.DATA_DIR}/debug_meta_train/common_date_data/valid.jsonl")

In [ ]:

ds = load_dataset("ucinlp/drop")

date_count = 0
failed_count = 0
drop_date_instances = []
passage = set()
for split in ["train", "validation"]:
    for i in range(len(ds[split])):
        datum = ds[split][i]
        span = datum["answers_spans"]
        a_str = datum["answers_spans"]["spans"][0]
        
        q_str = datum["question"].lower()
        date_count += len([t for t in span["types"] if t == "date"])
        
        if not any([t in q_str.lower() for t in ["date", "year", "when",]]):
            continue
        if any([t in q_str.lower() for t in ["month", "how many", "how old"]]):
            continue
        
        if "date" in span["types"] and datum["passage"] not in passage:
            date_index = span["types"].index("date")
            a_str = span["spans"][date_index]
            
            try:
                if str(parse(a_str).year) in a_str:
                    drop_date_instances.append(datum)
            except:
                failed_count += 1
                pass

drop_unified_format = []

for datum in drop_date_instances:
    drop_unified_format.append({
        "id": datum["query_id"],
        "question": datum["question"],
        "answer": datum["answers_spans"]["spans"][0],
        # "texts": [],
        "dataset": "drop"
    })
len(drop_unified_format)

1097

In [ ]:
musique_date_instances = []
single_hop_id = set()
years = []

for split in ["train", "dev", "test"]:
    musique_instances = io.load_jsonlines(f"{vars.DATA_DIR}/musique/musique_ans_v1.0_{split}.jsonl")

    for instance in musique_instances:
        if "question_decomposition" not in instance:
            continue
        for q in instance["question_decomposition"]:
            q_str = q["question"].lower()
            a_str = q["answer"]
            # if not any([t in q_str.lower() for t in ["date", "year", "when",]]):
            #     continue
            # if any([t in q_str.lower() for t in ["month", "how many", ]]):
            #     continue
            # if any([t in q_str for t in ["date", "year", "when",]]) and is_date(a_str) and q["id"] not in single_hop_id:
            if is_date(a_str) and q["id"] not in single_hop_id and str(parse(a_str).year) in a_str:
                single_hop_id.add(q["id"])
                years.append(parse(a_str).year)
                musique_date_instances.append(instance)

In [256]:
q_str

'how were the #1 expelled from #2 ?'

In [366]:
from datasets import load_dataset

ds = load_dataset("mandarjoshi/trivia_qa", "rc.nocontext")

questions = set()
c = 0
trivia_date_data = []
for split in ["train", "validation", "test"]:
     for i in range(len(ds[split])):
        datum = ds[split][i]
        a_str = datum["answer"]["value"]
        q_str = datum["question"]
        if not any([t in q_str.lower() for t in ["date", "year", "when",]]):
            continue
        if any([t in q_str.lower() for t in ["month", "how many", "how old"]]):
            continue
        if is_date(a_str) and q_str not in questions and str(parse(a_str).year) in a_str:
            questions.add(q_str)
            trivia_date_data.append(datum)
            c += 1
            
trivia_unified_format = []

for datum in trivia_date_data:
    trivia_unified_format.append({
        "id": datum["question_id"],
        "question": datum["question"],
        "answer": datum["answer"]["value"],
        # "texts": [],
        "dataset": "triviaqa"
    })
len(trivia_unified_format)

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

1075

In [ ]:
from tqdm import tqdm
questions = set()
c = 0
hotpotqa_date_data = []
# for split in ["train", "validation", "test"]:
for split, filename in [("train", "hotpot_train_v1.1.json"), ("dev", "hotpot_dev_fullwiki_v1.json"), ("test", "hotpot_test_fullwiki_v1.json")]:
    data = io.load_json(f"{vars.DATA_DIR}/hotpotqa/{filename}")
    for i in tqdm(range(len(data))):
        datum = data[i]
        if "answer" not in datum or "question" not in datum:
            continue
        a_str = datum["answer"]
        q_str = datum["question"]
        if not any([t in q_str.lower() for t in ["date", "year", "when",]]):
            continue
        if any([t in q_str.lower() for t in ["month", "how many", "how old"]]):
            continue
        if is_date(a_str) and q_str not in questions and str(parse(a_str).year) in a_str:
            questions.add(q_str)
            hotpotqa_date_data.append(datum)
            c += 1
            
hotpotqa_unified_format = []

for datum in hotpotqa_date_data:
    supporting_titles = [t for t, _ in datum["supporting_facts"]]
    texts = ["".join(lines) for t, lines in datum["context"] if t in supporting_titles]
    
    hotpotqa_unified_format.append({
        "id": datum["_id"],
        "question": datum["question"],
        "answer": datum["answer"],
        # "texts": texts,
        "dataset": "hotpotqa"
    })
len(hotpotqa_unified_format)

100%|██████████| 7405/7405 [00:00<00:00, 2338238.43it/s]


9517

In [ ]:
bio_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data/train.jsonl")

NameError: name 'io' is not defined

9518

In [361]:
hotpotqa_unified_format[0]

{'id': '5a7d0db955429909bec76924',
 'question': 'The Dutch-Belgian television series that "House of Anubis" was based on first aired in what year?',
 'answer': '2006',
 'dataset': 'hotpotqa'}

In [ ]:
sft_data = trivia_unified_format + hotpotqa_unified_format + drop_unified_format


pd.DataFrame([d for d in sft_data if len(d["question"]) <= 400]).to_excel(f"{vars.DATA_DIR}/debug_meta_train/real_date_data/all_data.xlsx", index=False)

In [ ]:
filter_sft_data = []
special_data = []
for datum in sft_data:
    if datum["answer"].isdigit() and len(datum["answer"]) != 4:
        special_data.append(datum)

In [377]:
from scipy.stats import describe

pd.DataFrame([len(d["question"]) for d in sft_data]).describe()

,0
count,11689.000000
mean,107.059201
std,64.366637
min,17.000000
25%,69.000000
50%,90.000000
75%,122.000000
max,654.000000


In [ ]:
rewrite_data = pd.read_excel(f"{vars.DATA_DIR}/debug_meta_train/real_date_data/all_data_w-rewrite.xlsx").to_dict(orient="records")

In [419]:
increment_rewrite_data = []
failed_count = 0
for datum in rewrite_data:
    a_str = datum["answer"]
    try:
        assert str(parse(a_str).year) in a_str
        rewrite_answer = str(int(parse(a_str).year) + 1)
        increment_rewrite_datum = deepcopy(datum)
        
        increment_rewrite_datum["rewrite_answer"] = rewrite_answer
        increment_rewrite_data.append(increment_rewrite_datum)
    except:
        failed_count += 1
        continue
failed_count

5

In [ ]:
pd.DataFrame(increment_rewrite_data).to_excel(f"{vars.DATA_DIR}/debug_meta_train/real_date_data/all_data_w-rewrite_increment.xlsx", index=False)

In [ ]:
all_data = pd.read_excel(f"{vars.DATA_DIR}/debug_meta_train/real_date_data/all_data_w-rewrite_increment.xlsx").to_dict(orient="records")

In [ ]:
io.dump_jsonlines(all_data, f"{vars.DATA_DIR}/debug_meta_train/real_date_data/all_data_w-rewrite_increment.jsonl")

In [426]:
len([d for d in all_data if len(d["question"]) <= 200])

10785

In [ ]:
increment_date_data = [
    {
        "id": d["id"],
        "question": d["rewrite_question"],
        "answer": d["rewrite_answer"],
        "original_question": d["question"],
        "original_answer": d["answer"],
        "dataset": d["dataset"],
    } 
    for d in all_data ]

In [413]:
n_dev = 1000
n_train = len(increment_date_data) - n_dev

rand_shuffle = np.arange(len(increment_date_data))
np.random.shuffle(rand_shuffle)

increment_date_data_train = [increment_date_data[i] for i in rand_shuffle[:n_train]]
increment_date_data_dev = [increment_date_data[i] for i in rand_shuffle[n_train:]]

In [ ]:
io.dump_jsonlines(increment_date_data_train, f"{vars.DATA_DIR}/debug_meta_train/real_date_data/increment_date_data_train.jsonl")
io.dump_jsonlines(increment_date_data_dev, f"{vars.DATA_DIR}/debug_meta_train/real_date_data/increment_date_data_dev.jsonl")

# Generate synthetic year meta-training data

In [3]:
text_template = "{first_name} {last_name} was born in {birth_year} in {birth_place}. {gender_start} started the career of {career} in {career_year}. In {death_year}, {gender} passed away."

In [2]:
first_names = list(set("""Michael
Emma
James
Sophia
David
Olivia
William
Ava
Alexander
Isabella
John
Mia
Matthew
Charlotte
Daniel
Amelia
Christopher
Harper
Joseph
Evelyn
Benjamin
Abigail
Andrew
Emily
Robert
Elizabeth
Thomas
Sofia
Samuel
Avery
Jacob
Ella
Nathan
Scarlett
Nicholas
Grace
Ryan
Victoria
Joshua
Madison
Ethan
Lily
Noah
Hannah
Anthony
Chloe
Jonathan
Zoe
Aaron
Nora
Gabriel
Riley
Lucas
Layla
Christina
Maria
Jason
Sarah
Tyler
Natalie
Kevin
Leah
Eric
Maya
Brian
Jennifer
Brandon
Laura
Adam
Elena
Marcus
Jasmine
Caleb
Anna""".split("\n")))

In [3]:
last_names = list(set("""Smith
Johnson
Williams
Brown
Jones
Garcia
Miller
Davis
Rodriguez
Martinez
Hernandez
Lopez
Gonzalez
Wilson
Anderson
Thomas
Taylor
Moore
Jackson
Martin
Lee
Perez
Thompson
White
Harris
Sanchez
Clark
Ramirez
Lewis
Robinson
Walker
Young
Allen
King
Wright
Scott
Torres
Nguyen
Hill
Flores
Green
Adams
Nelson
Baker
Hall
Rivera
Campbell
Mitchell
Carter
Roberts
Gomez
Phillips
Evans
Turner
Diaz
Parker
Cruz
Edwards
Collins
Reyes
Stewart
Morris
Morales
Murphy
Cook
Rogers
Gutierrez
Ortiz
Morgan
Cooper
Peterson
Bailey
Reed
Kelly
Howard
Ramos
Kim
Cox
Ward
Richardson
Watson
Brooks
Chavez
Wood
James
Bennett
Gray
Mendoza
Ruiz
Hughes
Price
Alvarez
Castillo""".split("\n")))

In [4]:
genders = ["he", "she"]
careers = list(set("""Doctor
Teacher
Software Engineer
Nurse
Accountant
Chef
Architect
Lawyer
Electrician
Marketing Manager
Graphic Designer
Pharmacist
Police Officer
Financial Analyst
Journalist
Mechanical Engineer
Social Worker
Veterinarian
Pilot
Dental Hygienist
Web Developer
Physical Therapist
Human Resources Manager
Firefighter
Real Estate Agent
Data Scientist
Interior Designer
Occupational Therapist
Construction Manager
Speech Pathologist
Cybersecurity Analyst
Photographer
Psychologist
Plumber
Flight Attendant
Marine Biologist
Athletic Trainer
Urban Planner
Welder
Dietitian
Librarian
Civil Engineer
Paralegal
Film Producer
Actuary
Event Planner
Scientist
Carpenter
Financial Advisor
Lab Technician""".split("\n")))

In [46]:
birth_years = np.random.choice(range(1900, 2020), 100, replace=False)
birth_places = list(set("""New York City
Tokyo
Paris
London
Sydney
Rome
Barcelona
Amsterdam
Singapore
Cairo
Rio de Janeiro
Vancouver
Istanbul
Dubai
Cape Town
Bangkok
Dublin
Seoul
Venice
Hong Kong
San Francisco
Mumbai
Berlin
Buenos Aires
Prague
Stockholm
Montreal
Beijing
Athens
Marrakech
Kyoto
Vienna
Copenhagen
Jerusalem
Nairobi
Mexico City
Santorini
Bali
Toronto
Reykjavik
Havana
Moscow
Queenstown
Florence
Edinburgh
Machu Picchu
Petra
Bora Bora
Chicago
Maldives""".split("\n")))

In [1]:
len(first_names) * len(last_names) * len(birth_years) * len(birth_places) * len(genders) * len(careers)

NameError: name 'first_names' is not defined

In [9]:

n_data = 1000000

tuples = set()

career_year_question_template = "When was the year after the year that {first_name} {last_name} started the career of {career}?"
death_year_question_template = "When was the year after the year that {first_name} {last_name} passed away?"
birth_year_question_template = "When was the year after the year that {first_name} {last_name} was born?"

syn_data = []

pbar = tqdm(total = n_data)

max_death_year = 2023


while len(syn_data) < n_data:
    
    first_name = np.random.choice(first_names)
    last_name = np.random.choice(last_names)


    birth_year = np.random.choice(birth_years)
    birth_place = np.random.choice(birth_places)
    gender = np.random.choice(genders)
    career = np.random.choice(careers)

    growth_duration = np.random.randint(14, 26)
    career_year = birth_year + growth_duration
    career_duration = np.random.randint(4, 40)
    retire_year = career_year + career_duration

    retire_duration = np.random.randint(1, 10)
    death_year = retire_year + retire_duration
    if death_year > max_death_year:
        continue
    
    info_tuple = (first_name, last_name, birth_year, birth_place, gender, career, career_year, death_year)
    
    if info_tuple not in tuples:
        tuples.add(info_tuple)
    else:
        continue

    text = text_template.format(
        first_name=first_name,
        last_name=last_name,
        birth_year=birth_year,
        birth_place=birth_place,
        gender=gender,
        gender_start=gender.capitalize(),
        career=career,
        career_year=career_year,
        death_year=death_year,
    )
    
    rand_idx = np.random.choice(range(3))

    template_name, question_template = [
        ("career", career_year_question_template), 
        ("death",death_year_question_template), 
        ("birth", birth_year_question_template),
    ][rand_idx]
    
    question = question_template.format(
        first_name=first_name,
        last_name=last_name,
        career=career,
    )
    if template_name == "career":
        answer = career_year + 1
    elif template_name == "death":
        answer = death_year + 1
    else:
        answer = birth_year + 1

    syn_data.append(
        {
            "text": text,
            "question": question,
            "answer": str(int(answer))
        }
    )
    pbar.update(1)
        

100%|█████████▉| 999560/1000000 [02:03<00:00, 8039.46it/s]

In [ ]:
n_test = 100
n_dev = 100
n_train = n_data - n_test - n_dev

rand_shuffle = np.arange(n_data)
np.random.shuffle(rand_shuffle)

train_data = [syn_data[i] for i in rand_shuffle[:n_train]]
dev_data = [syn_data[i] for i in rand_shuffle[n_train:n_train+n_dev]]
test_data = [syn_data[i] for i in rand_shuffle[n_train+n_dev:]]

io.dump_jsonlines(train_data, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data/train.jsonl")
io.dump_jsonlines(dev_data, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data/valid.jsonl")
io.dump_jsonlines(test_data, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data/test.jsonl")

In [10]:
io.dump_jsonlines(syn_data, "/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/bio_syn_data/all_data.jsonl")

100%|██████████| 1000000/1000000 [02:20<00:00, 8039.46it/s]

In [48]:
f = lambda *x: x[1] + 1

In [49]:
f(1,2,3)

3

### Bio data with more template (v2)

In [4]:
events = [
    "{first_name} {last_name} was born",
    "{first_name} {last_name} started the career of {career}", 
    "{first_name} {last_name} passed away",
]

In [ ]:
# year offset template
# - 1
# 
# plus_1_templates = [
#     ("When was the year after the year that" + " " + e + "?", lambda *x: x[e_i] + 1)
#     for e_i, e in enumerate(events)
# ]
plus_1_templates = [
    ("When was the year after the year that" + " " + events[0] + "?", lambda *x: x[0] + 1),
    ("When was the year after the year that" + " " + events[1] + "?", lambda *x: x[1] + 1),
    ("When was the year after the year that" + " " + events[2] + "?", lambda *x: x[2] + 1),
]
minus_1_templates = [
    ("When was the year before the year that" + " " + events[0] + "?", lambda *x: x[0] - 1),
    ("When was the year before the year that" + " " + events[1] + "?", lambda *x: x[1] - 1),
    ("When was the year before the year that" + " " + events[2] + "?", lambda *x: x[2] - 1),
]

plus_3_templates = [
    ("When was 3 year after the year that" + " " + events[0] + "?", lambda *x: x[0] + 3),
    ("When was 3 year after the year that" + " " + events[1] + "?", lambda *x: x[1] + 3),
    ("When was 3 year after the year that" + " " + events[2] + "?", lambda *x: x[2] + 3),
]
minus_3_templates = [
    ("When was 3 year before the year that" + " " + events[0] + "?", lambda *x: x[0] - 3),
    ("When was 3 year before the year that" + " " + events[1] + "?", lambda *x: x[1] - 3),
    ("When was 3 year before the year that" + " " + events[2] + "?", lambda *x: x[2] - 3),
]


plus_10_templates = [
    ("When was 10 year after the year that" + " " + events[0] + "?", lambda *x: x[0] + 10),
    ("When was 10 year after the year that" + " " + events[1] + "?", lambda *x: x[1] + 10),
    ("When was 10 year after the year that" + " " + events[2] + "?", lambda *x: x[2] + 10),
]
minus_10_templates = [
    ("When was 10 year before the year that" + " " + events[0] + "?", lambda *x: x[0] - 10),
    ("When was 10 year before the year that" + " " + events[1] + "?", lambda *x: x[1] - 10),
    ("When was 10 year before the year that" + " " + events[2] + "?", lambda *x: x[2] - 10),
]

interval_templates = [
    ("How many years did {first_name} {last_name} live for?", lambda b,c,d: d-b),
    ("How many years after {first_name} {last_name} started the career of {career} did {gender} pass away?", lambda b,c,d: d-c),
    ("How many years after {first_name} {last_name} was born did {gender} start the career of {career}?", lambda b,c,d: c-b),
]

In [ ]:

n_data = 100000

tuples = set()

syn_data = []

pbar = tqdm(total = n_data)

max_death_year = 2023


while len(syn_data) < n_data:
    
    first_name = np.random.choice(first_names)
    last_name = np.random.choice(last_names)


    birth_year = np.random.choice(birth_years)
    birth_place = np.random.choice(birth_places)
    gender = np.random.choice(genders)
    career = np.random.choice(careers)

    growth_duration = np.random.randint(14, 26)
    career_year = birth_year + growth_duration
    career_duration = np.random.randint(4, 40)
    retire_year = career_year + career_duration

    retire_duration = np.random.randint(1, 10)
    death_year = retire_year + retire_duration
    if death_year > max_death_year:
        continue
    
    info_tuple = (first_name, last_name, birth_year, birth_place, gender, career, career_year, death_year)
    
    if info_tuple not in tuples:
        tuples.add(info_tuple)
    else:
        continue

    text = text_template.format(
        first_name=first_name,
        last_name=last_name,
        birth_year=birth_year,
        birth_place=birth_place,
        gender=gender,
        gender_start=gender.capitalize(),
        career=career,
        career_year=career_year,
        death_year=death_year,
    )
    
    questions = []
    for template_name, templates in [
        ("plus_1", plus_1_templates) , 
        ("minus_1", minus_1_templates), 
        ("plus_3", plus_3_templates), 
        ("minus_3", minus_3_templates), 
        ("plus_10", plus_10_templates),
        ("minus_10", minus_10_templates),
        ("interval", interval_templates),
    ]:
        
        rand_idx = np.random.choice(range(len(templates)))

        question_template, answer_fn = templates[rand_idx]
    
        question = question_template.format(
            first_name=first_name,
            last_name=last_name,
            career=career,
            gender=gender
        )
        answer = answer_fn(birth_year, career_year, death_year)
        questions.append(
            {
                "question_type": template_name,
                "question": question,
                "answer": str(int(answer))
            }
        )
    syn_data.append(
        {
            "text": text,
            "questions": questions
        }
    )
    pbar.update(1)
        

100%|█████████▉| 99615/100000 [00:18<00:00, 5440.39it/s]

In [113]:
text

'Anna Rivera was born in 1980 in Sydney. She started the career of Doctor in 2003. In 2020, she passed away.'

In [106]:
plus_1_templates[2][0].format(
            first_name=first_name,
            last_name=last_name,
            career=career,
            gender=gender
        )

'When was the year after the year that Matthew Evans passed away?'

In [107]:
plus_1_templates

[('When was the year after the year that {first_name} {last_name} was born?',
  <function __main__.<lambda>(*x)>),
 ('When was the year after the year that {first_name} {last_name} started the career of {career}?',
  <function __main__.<lambda>(*x)>),
 ('When was the year after the year that {first_name} {last_name} passed away?',
  <function __main__.<lambda>(*x)>)]

In [108]:
plus_1_templates[2][1](birth_year, career_year, death_year)

np.int64(1973)

#### Parse synthetic text and generate QA from another template per template type

In [ ]:
template_type2templates = dict([
    ("plus_1", plus_1_templates) , 
    ("minus_1", minus_1_templates), 
    ("plus_3", plus_3_templates), 
    ("minus_3", minus_3_templates), 
    ("plus_10", plus_10_templates),
    ("minus_10", minus_10_templates),
    ("interval", interval_templates),
])
test_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test.jsonl")

In [10]:


# Define the regex pattern
pattern = r"(?P<first_name>\w+) (?P<last_name>\w+) was born in (?P<birth_year>\d{4}) in (?P<birth_place>[\w\s,]+)\. (?P<gender_start>\w+) started the career of (?P<career>[\w\s]+) in (?P<career_year>\d{4})\. In (?P<death_year>\d{4}), (?P<gender>\w+) passed away\."

# Example function to extract information
def extract_person_info(text):
    match = re.search(pattern, text)
    if match:
        return match.groupdict()
    return None

# Example usage
# text = "John Doe was born in 1950 in New York City. He started the career of engineering in 1972. In 2020, he passed away."
# info = extract_person_info(text)
# print(info)

In [ ]:
test_data[0]["text"]

'Laura Roberts was born in 1937 in San Francisco. She started the career of Physical Therapist in 1955. In 1993, she passed away.'

'How many years after Laura Roberts was born did she start the career of Physical Therapist?'

In [24]:
test_data[0]["questions"]

[{'question_type': 'plus_1',
  'question': 'When was the year after the year that Laura Roberts started the career of Physical Therapist?',
  'answer': '1956'},
 {'question_type': 'minus_1',
  'question': 'When was the year before the year that Laura Roberts was born?',
  'answer': '1936'},
 {'question_type': 'plus_3',
  'question': 'When was 3 year after the year that Laura Roberts was born?',
  'answer': '1940'},
 {'question_type': 'minus_3',
  'question': 'When was 3 year before the year that Laura Roberts passed away?',
  'answer': '1990'},
 {'question_type': 'plus_10',
  'question': 'When was 10 year after the year that Laura Roberts was born?',
  'answer': '1947'},
 {'question_type': 'minus_10',
  'question': 'When was 10 year before the year that Laura Roberts was born?',
  'answer': '1927'},
 {'question_type': 'interval',
  'question': 'How many years did Laura Roberts live for?',
  'answer': '56'}]

In [88]:
new_test_data = []

for test_datum in test_data: 
    new_test_datum = deepcopy(test_datum)
    syn_info = extract_person_info(test_datum["text"])
    
    alternative_questions = []
    for q in test_datum["questions"]:
        templates = template_type2templates[q["question_type"]]
        instantiated_templates = [
            template[0].format(first_name=syn_info["first_name"], last_name=syn_info["last_name"], career=syn_info["career"],gender=syn_info["gender"])
            for template in templates
        ]
        
        assert len([e_i for e_i, t in enumerate(instantiated_templates) if q["question"] in t]) == 1
        assert len([e_i for e_i, t in enumerate(instantiated_templates) if q["question"] not in t]) == 2
        
        alternative_template_ids = [e_i for e_i, t in enumerate(instantiated_templates) if q["question"] not in t]
        alternative_template_idx = np.random.choice(alternative_template_ids)
        alternative_template = templates[alternative_template_idx]
        alternative_questions.append(
            {
                "question_type": q["question_type"],
                "question": alternative_template[0].format(
                    first_name=syn_info["first_name"],
                    last_name=syn_info["last_name"],
                    career=syn_info["career"],
                    gender=syn_info["gender"],
                ),
                "answer": str(alternative_template[1](int(syn_info["birth_year"]), int(syn_info["career_year"]), int(syn_info["death_year"])))
            }
        )
    new_test_datum["alternative_questions"] = deepcopy(alternative_questions)
    
    np.random.shuffle(alternative_questions)
    text_w_qas = test_datum["text"] + "\n\n" + "\n".join(f"{q['question']} {q['answer']}" for q in alternative_questions)
    new_test_datum["text_w_qas"] = text_w_qas
    
    new_test_data.append(new_test_datum)

In [89]:
print(text_w_qas)

Kevin Baker was born in 1921 in Stockholm. He started the career of Film Producer in 1942. In 1966, he passed away.

When was 10 year before the year that Kevin Baker was born? 1911
When was the year after the year that Kevin Baker started the career of Film Producer? 1943
When was 10 year after the year that Kevin Baker started the career of Film Producer? 1952
How many years after Kevin Baker started the career of Film Producer did he pass away? 24
When was 3 year before the year that Kevin Baker passed away? 1963
When was 3 year after the year that Kevin Baker was born? 1924
When was the year before the year that Kevin Baker was born? 1920


In [91]:
io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test.jsonl")

In [37]:
[e_i for e_i, e in enumerate(syn_events) if e in test_data[0]["questions"][0]]

[]

In [44]:
q

{'question_type': 'interval',
 'question': 'How many years did Laura Roberts live for?',
 'answer': '56'}

In [47]:
[e in test_data[0]["questions"][0]["question"] for e_i, e in enumerate(syn_events)]

[False, True, False]

In [46]:
syn_events

['Laura Roberts was born',
 'Laura Roberts started the career of Physical Therapist',
 'Laura Roberts passed away']

In [42]:
syn_events[1] in test_data[0]["questions"][0]["question"]

True

In [115]:
n_test = 100
n_dev = 100
n_train = n_data - n_test - n_dev

rand_shuffle = np.arange(n_data)
np.random.shuffle(rand_shuffle)

train_data = [syn_data[i] for i in rand_shuffle[:n_train]]
dev_data = [syn_data[i] for i in rand_shuffle[n_train:n_train+n_dev]]
test_data = [syn_data[i] for i in rand_shuffle[n_train+n_dev:]]

io.dump_jsonlines(train_data, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/train.jsonl")
io.dump_jsonlines(dev_data, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/valid.jsonl")
io.dump_jsonlines(test_data, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test.jsonl")

100

In [ ]:
pd.DataFrame(io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data/train.jsonl")).sample(10).to_excel("/u/zliu/datastor1/mend/spotcheck/syn_text_train_sample.xlsx", index=False)

In [ ]:
pd.DataFrame(io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/common_date_data/train.jsonl")).sample(10).to_excel("/u/zliu/datastor1/mend/spotcheck/common_fact_train_sample.xlsx", index=False)

In [2]:
career_year_question_template = "When was the year after the year that {first_name} {last_name} started the career of {career}?"
death_year_question_template = "When was the year after the year that {first_name} {last_name} passed away?"
birth_year_question_template = "When was the year after the year that {first_name} {last_name} was born?"

In [10]:
def extract_info_birth(question):
    # Define the pattern to match the template
    pattern = r"When was the year after the year that ([\w-]+) ([\w-]+) was born\?"
    # Search for the pattern in the question
    match = re.search(pattern, question)
    
    if match:
        # Extract the captured groups
        first_name = match.group(1)
        last_name = match.group(2)
        
        return {
            "first_name": first_name,
            "last_name": last_name,
        }
    else:
        return None
    
def extract_info_death(question):
    # Define the pattern to match the template
    pattern = r"When was the year after the year that ([\w-]+) ([\w-]+) passed away\?"
    # Search for the pattern in the question
    match = re.search(pattern, question)
    
    if match:
        # Extract the captured groups
        first_name = match.group(1)
        last_name = match.group(2)
        
        return {
            "first_name": first_name,
            "last_name": last_name,
        }
    else:
        return None

def extract_info_career(question):
    # Define the pattern to match the template
    pattern = r"When was the year after the year that ([\w-]+) ([\w-]+) started the career of ([\w\s-]+)\?"
    
    # Search for the pattern in the question
    match = re.search(pattern, question)
    
    if match:
        # Extract the captured groups
        first_name = match.group(1)
        last_name = match.group(2)
        career = match.group(3)
        
        return {
            "first_name": first_name,
            "last_name": last_name,
            "career": career
        }
    else:
        return None

In [6]:
test_examples = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data/test.jsonl")

In [9]:
extract_info_career(test_examples[0]["question"])

{'first_name': 'Tyler', 'last_name': 'Ortiz', 'career': 'Pharmacist'}

In [12]:
extract_info_death(test_examples[1]["question"])

{'first_name': 'Nicholas', 'last_name': 'Reyes'}

In [16]:
extract_info_birth(test_examples[4]["question"])

{'first_name': 'William', 'last_name': 'Wilson'}

In [17]:
test_examples_n_question = []

career_year_n_question_template = "When was the year that {first_name} {last_name} started the career of {career}?"
death_year_n_question_template = "When was the year that {first_name} {last_name} passed away?"
birth_year_n_question_template = "When was the year that {first_name} {last_name} was born?"

for example in test_examples:
    new_example = deepcopy(example)
    
    if "career" in example["question"]:
        new_example["info"] = extract_info_career(example["question"])
        new_template = career_year_n_question_template
    elif "passed away" in example["question"]:
        new_example["info"] = extract_info_death(example["question"])
        new_template = death_year_n_question_template
    else:
        assert "was born" in example["question"]
        new_example["info"] = extract_info_birth(example["question"])
        new_template = birth_year_n_question_template

    new_example["question"] = new_template.format(**new_example["info"])
    new_example["answer"] = str(int(new_example["answer"]) - 1)
    
    test_examples_n_question.append(new_example)

In [19]:
io.dump_jsonlines(test_examples_n_question, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data/test_n_question.jsonl")

# Generate place meta-training data

In [5]:
import geonamescache

ModuleNotFoundError: No module named 'geonamescache'

In [5]:
gc = geonamescache.GeonamesCache()
countries = gc.get_countries()
countries_by_population = sorted(countries.values(), key=lambda x: -x["population"])
print("# total country:", len(countries_by_population))

countries_by_population_top20 = countries_by_population[:20]
countries_by_population_top20_country_code = set([c["iso"] for c in countries_by_population_top20])

# total country: 252


In [5]:
countries_by_population[-1]

{'geonameid': 5854968,
 'name': 'United States Minor Outlying Islands',
 'iso': 'UM',
 'iso3': 'UMI',
 'isonumeric': 581,
 'fips': '',
 'continentcode': 'OC',
 'capital': '',
 'areakm2': 0,
 'population': 0,
 'tld': '.um',
 'currencycode': 'USD',
 'currencyname': 'Dollar',
 'phone': '1',
 'postalcoderegex': '',
 'languages': 'en-UM',
 'neighbours': ''}

In [6]:
cities_by_population = sorted(gc.get_cities().values(), key=lambda x: -x["population"])

print("# total cities:", len(cities_by_population))

# total cities: 26463


In [10]:
countrycode2cities = defaultdict(list)
for city in cities_by_population:
    # if city["countrycode"] in countries_by_population_top20_country_code:
    countrycode2cities[city["countrycode"]].append(city)
countrycode2cities = {cc: sorted(cities, key=lambda x: -x["population"]) for cc, cities in countrycode2cities.items()}
# countrycode2cities = {cc: cities for cc, cities in countrycode2cities.items() if len(cities) >= 20}


In [28]:
countrycode_by_n_cities = sorted([(cc, len(cities)) for cc, cities in countrycode2cities.items()], key=lambda x: -x[1])
len(countrycode_by_n_cities)

244

In [27]:
sorted([(countries[cc]["name"], len(cities)) for cc, cities in countrycode2cities.items()], key=lambda x: -x[1])

[('United States', 3272),
 ('India', 2480),
 ('China', 1955),
 ('Brazil', 1217),
 ('Germany', 1117),
 ('Russia', 1095),
 ('Japan', 851),
 ('United Kingdom', 829),
 ('France', 649),
 ('Mexico', 640),
 ('Spain', 625),
 ('Italy', 613),
 ('Philippines', 460),
 ('Indonesia', 402),
 ('Turkey', 392),
 ('Poland', 333),
 ('Canada', 330),
 ('Pakistan', 317),
 ('Colombia', 312),
 ('Australia', 290),
 ('Ukraine', 255),
 ('Algeria', 250),
 ('Nigeria', 245),
 ('Argentina', 229),
 ('Netherlands', 228),
 ('Belgium', 224),
 ('Thailand', 222),
 ('Venezuela', 210),
 ('Iran', 205),
 ('Tanzania', 185),
 ('South Africa', 170),
 ('Malaysia', 142),
 ('Portugal', 141),
 ('Romania', 134),
 ('Egypt', 133),
 ('Peru', 131),
 ('Cuba', 127),
 ('South Korea', 126),
 ('Vietnam', 116),
 ('Hungary', 114),
 ('Greece', 114),
 ('Sweden', 106),
 ('Bangladesh', 105),
 ('Chile', 103),
 ('Uzbekistan', 103),
 ('Israel', 103),
 ('Guatemala', 102),
 ('Czechia', 96),
 ('Ethiopia', 91),
 ('Switzerland', 86),
 ('Morocco', 85),
 ('Ma

In [33]:
sft_cities = []
for i, (cc, _) in enumerate(countrycode_by_n_cities):
    if i < 10:
        sft_cities.extend(countrycode2cities[cc][:80])
    elif i < 30:
        sft_cities.extend(countrycode2cities[cc][:30])
    elif i < 70:
        sft_cities.extend(countrycode2cities[cc][:10])
    elif i < 150:
        sft_cities.extend(countrycode2cities[cc][:4])
    else:
        continue
    

In [21]:
countrycode_by_n_cities

[('US', 3272),
 ('IN', 2480),
 ('CN', 1955),
 ('BR', 1217),
 ('DE', 1117),
 ('RU', 1095),
 ('JP', 851),
 ('GB', 829),
 ('FR', 649),
 ('MX', 640),
 ('ES', 625),
 ('IT', 613),
 ('PH', 460),
 ('ID', 402),
 ('TR', 392),
 ('PL', 333),
 ('CA', 330),
 ('PK', 317),
 ('CO', 312),
 ('AU', 290),
 ('UA', 255),
 ('DZ', 250),
 ('NG', 245),
 ('AR', 229),
 ('NL', 228),
 ('BE', 224),
 ('TH', 222),
 ('VE', 210),
 ('IR', 205),
 ('TZ', 185),
 ('ZA', 170),
 ('MY', 142),
 ('PT', 141),
 ('RO', 134),
 ('EG', 133),
 ('PE', 131),
 ('CU', 127),
 ('KR', 126),
 ('VN', 116),
 ('HU', 114),
 ('GR', 114),
 ('SE', 106),
 ('BD', 105),
 ('CL', 103),
 ('UZ', 103),
 ('IL', 103),
 ('GT', 102),
 ('CZ', 96),
 ('ET', 91),
 ('CH', 86),
 ('MA', 85),
 ('MG', 84),
 ('KZ', 78),
 ('SY', 76),
 ('FI', 76),
 ('TN', 74),
 ('IQ', 72),
 ('CD', 68),
 ('MM', 67),
 ('EC', 66),
 ('SA', 63),
 ('CM', 63),
 ('GH', 61),
 ('CI', 60),
 ('KP', 59),
 ('DK', 59),
 ('AZ', 59),
 ('KE', 58),
 ('BG', 56),
 ('NZ', 55),
 ('LK', 54),
 ('SK', 53),
 ('AF', 49)

In [29]:
10 * 80 + 20 * 30 + 40 * 10 + 80 * 4

2120

### SFT data for country

In [6]:
cities_1 = [
    ("New York", "USA"),
    ("London", "United Kingdom"),
    ("Tokyo", "Japan"),
    ("Paris", "France"),
    ("Berlin", "Germany"),
    ("Madrid", "Spain"),
    ("Rome", "Italy"),
    ("Toronto", "Canada"),
    ("Sydney", "Australia"),
    ("Moscow", "Russia"),
    ("Beijing", "China"),
    ("Mumbai", "India"),
    ("São Paulo", "Brazil"),
    ("Mexico City", "Mexico"),
    ("Cairo", "Egypt"),
    ("Istanbul", "Turkey"),
    ("Bangkok", "Thailand"),
    ("Buenos Aires", "Argentina"),
    ("Seoul", "South Korea"),
    ("Jakarta", "Indonesia"),
    ("Dubai", "United Arab Emirates"),
    ("Singapore", "Singapore"),
    ("Hong Kong", "China"),
    ("Lagos", "Nigeria"),
    ("Johannesburg", "South Africa"),
    ("Tehran", "Iran"),
    ("Lima", "Peru"),
    ("Bogotá", "Colombia"),
    ("Hanoi", "Vietnam"),
    ("Kuala Lumpur", "Malaysia"),
    ("Riyadh", "Saudi Arabia"),
    ("Vienna", "Austria"),
    ("Athens", "Greece"),
    ("Dublin", "Ireland"),
    ("Oslo", "Norway"),
    ("Stockholm", "Sweden"),
    ("Copenhagen", "Denmark"),
    ("Brussels", "Belgium"),
    ("Amsterdam", "Netherlands"),
    ("Helsinki", "Finland"),
    ("Zurich", "Switzerland"),
    ("Warsaw", "Poland"),
    ("Budapest", "Hungary"),
    ("Prague", "Czech Republic"),
    ("Lisbon", "Portugal"),
    ("Edinburgh", "United Kingdom"),
    ("Glasgow", "United Kingdom"),
    ("Barcelona", "Spain"),
    ("Milan", "Italy"),
    ("Geneva", "Switzerland"),
    ("Munich", "Germany"),
    ("Frankfurt", "Germany"),
    ("Hamburg", "Germany"),
    ("Brisbane", "Australia"),
    ("Melbourne", "Australia"),
    ("Perth", "Australia"),
    ("Wellington", "New Zealand"),
    ("Auckland", "New Zealand"),
    ("Christchurch", "New Zealand"),
    ("Doha", "Qatar"),
    ("Manila", "Philippines"),
    ("Ho Chi Minh City", "Vietnam"),
    ("Taipei", "Taiwan"),
    ("Karachi", "Pakistan"),
    ("Dhaka", "Bangladesh"),
    ("Santiago", "Chile"),
    ("Montevideo", "Uruguay"),
    ("Quito", "Ecuador"),
    ("La Paz", "Bolivia"),
    ("Kuwait City", "Kuwait"),
    ("Muscat", "Oman"),
    ("Baku", "Azerbaijan"),
    ("Tbilisi", "Georgia"),
    ("Yerevan", "Armenia"),
    ("Casablanca", "Morocco"),
    ("Algiers", "Algeria"),
    ("Addis Ababa", "Ethiopia"),
    ("Nairobi", "Kenya"),
    ("Accra", "Ghana"),
    ("Kampala", "Uganda"),
    ("Harare", "Zimbabwe"),
    ("Kinshasa", "Democratic Republic of the Congo"),
    ("Lusaka", "Zambia"),
    ("Maputo", "Mozambique"),
    ("San Francisco", "USA"),
    ("Los Angeles", "USA"),
    ("Chicago", "USA"),
    ("Houston", "USA"),
    ("Miami", "USA"),
    ("Boston", "USA"),
    ("Dallas", "USA"),
    ("Seattle", "USA"),
    ("Atlanta", "USA"),
    ("Philadelphia", "USA"),
    ("Denver", "USA"),
    ("Phoenix", "USA"),
    ("Montreal", "Canada"),
    ("Vancouver", "Canada"),
    ("Calgary", "Canada"),
    ("Ottawa", "Canada"),
    ("Quebec City", "Canada"),
    ("Halifax", "Canada")
]

In [7]:
cities_2 = [
    ("New York", "USA"),
    ("Los Angeles", "USA"),
    ("Chicago", "USA"),
    ("Houston", "USA"),
    ("Miami", "USA"),
    ("London", "United Kingdom"),
    ("Manchester", "United Kingdom"),
    ("Birmingham", "United Kingdom"),
    ("Edinburgh", "United Kingdom"),
    ("Glasgow", "United Kingdom"),
    ("Toronto", "Canada"),
    ("Vancouver", "Canada"),
    ("Montreal", "Canada"),
    ("Calgary", "Canada"),
    ("Ottawa", "Canada"),
    ("Sydney", "Australia"),
    ("Melbourne", "Australia"),
    ("Brisbane", "Australia"),
    ("Perth", "Australia"),
    ("Adelaide", "Australia"),
    ("Paris", "France"),
    ("Marseille", "France"),
    ("Lyon", "France"),
    ("Toulouse", "France"),
    ("Nice", "France"),
    ("Berlin", "Germany"),
    ("Munich", "Germany"),
    ("Frankfurt", "Germany"),
    ("Hamburg", "Germany"),
    ("Cologne", "Germany"),
    ("Madrid", "Spain"),
    ("Barcelona", "Spain"),
    ("Valencia", "Spain"),
    ("Seville", "Spain"),
    ("Bilbao", "Spain"),
    ("Rome", "Italy"),
    ("Milan", "Italy"),
    ("Naples", "Italy"),
    ("Turin", "Italy"),
    ("Florence", "Italy"),
    ("Tokyo", "Japan"),
    ("Osaka", "Japan"),
    ("Kyoto", "Japan"),
    ("Nagoya", "Japan"),
    ("Sapporo", "Japan"),
    ("Beijing", "China"),
    ("Shanghai", "China"),
    ("Shenzhen", "China"),
    ("Guangzhou", "China"),
    ("Chengdu", "China"),
    ("Moscow", "Russia"),
    ("Saint Petersburg", "Russia"),
    ("Novosibirsk", "Russia"),
    ("Yekaterinburg", "Russia"),
    ("Kazan", "Russia"),
    ("São Paulo", "Brazil"),
    ("Rio de Janeiro", "Brazil"),
    ("Brasília", "Brazil"),
    ("Salvador", "Brazil"),
    ("Fortaleza", "Brazil"),
    ("Buenos Aires", "Argentina"),
    ("Córdoba", "Argentina"),
    ("Rosario", "Argentina"),
    ("Mendoza", "Argentina"),
    ("La Plata", "Argentina"),
    ("Mexico City", "Mexico"),
    ("Guadalajara", "Mexico"),
    ("Monterrey", "Mexico"),
    ("Puebla", "Mexico"),
    ("Tijuana", "Mexico"),
    ("Bangkok", "Thailand"),
    ("Chiang Mai", "Thailand"),
    ("Phuket", "Thailand"),
    ("Pattaya", "Thailand"),
    ("Ayutthaya", "Thailand"),
    ("Delhi", "India"),
    ("Mumbai", "India"),
    ("Bangalore", "India"),
    ("Kolkata", "India"),
    ("Chennai", "India"),
    ("Seoul", "South Korea"),
    ("Busan", "South Korea"),
    ("Incheon", "South Korea"),
    ("Daegu", "South Korea"),
    ("Daejeon", "South Korea"),
    ("Istanbul", "Turkey"),
    ("Ankara", "Turkey"),
    ("Izmir", "Turkey"),
    ("Bursa", "Turkey"),
    ("Antalya", "Turkey"),
    ("Cairo", "Egypt"),
    ("Alexandria", "Egypt"),
    ("Giza", "Egypt"),
    ("Luxor", "Egypt"),
    ("Aswan", "Egypt"),
    ("Johannesburg", "South Africa"),
    ("Cape Town", "South Africa"),
    ("Durban", "South Africa"),
    ("Pretoria", "South Africa"),
    ("Port Elizabeth", "South Africa")
]


In [8]:
cities_3 = [
    ("New York", "USA"),
    ("Los Angeles", "USA"),
    ("Chicago", "USA"),
    ("Miami", "USA"),
    ("San Francisco", "USA"),
    ("London", "United Kingdom"),
    ("Manchester", "United Kingdom"),
    ("Birmingham", "United Kingdom"),
    ("Edinburgh", "United Kingdom"),
    ("Glasgow", "United Kingdom"),
    ("Toronto", "Canada"),
    ("Vancouver", "Canada"),
    ("Montreal", "Canada"),
    ("Calgary", "Canada"),
    ("Ottawa", "Canada"),
    ("Sydney", "Australia"),
    ("Melbourne", "Australia"),
    ("Brisbane", "Australia"),
    ("Perth", "Australia"),
    ("Adelaide", "Australia"),
    ("Paris", "France"),
    ("Marseille", "France"),
    ("Lyon", "France"),
    ("Nice", "France"),
    ("Toulouse", "France"),
    ("Berlin", "Germany"),
    ("Munich", "Germany"),
    ("Frankfurt", "Germany"),
    ("Hamburg", "Germany"),
    ("Cologne", "Germany"),
    ("Madrid", "Spain"),
    ("Barcelona", "Spain"),
    ("Valencia", "Spain"),
    ("Seville", "Spain"),
    ("Bilbao", "Spain"),
    ("Rome", "Italy"),
    ("Milan", "Italy"),
    ("Venice", "Italy"),
    ("Florence", "Italy"),
    ("Naples", "Italy"),
    ("Tokyo", "Japan"),
    ("Osaka", "Japan"),
    ("Kyoto", "Japan"),
    ("Yokohama", "Japan"),
    ("Sapporo", "Japan"),
    ("Beijing", "China"),
    ("Shanghai", "China"),
    ("Hong Kong", "China"),
    ("Shenzhen", "China"),
    ("Guangzhou", "China"),
    ("Moscow", "Russia"),
    ("Saint Petersburg", "Russia"),
    ("Sochi", "Russia"),
    ("Yekaterinburg", "Russia"),
    ("Novosibirsk", "Russia"),
    ("São Paulo", "Brazil"),
    ("Rio de Janeiro", "Brazil"),
    ("Brasília", "Brazil"),
    ("Salvador", "Brazil"),
    ("Fortaleza", "Brazil"),
    ("Buenos Aires", "Argentina"),
    ("Córdoba", "Argentina"),
    ("Rosario", "Argentina"),
    ("Mendoza", "Argentina"),
    ("La Plata", "Argentina"),
    ("Mexico City", "Mexico"),
    ("Guadalajara", "Mexico"),
    ("Monterrey", "Mexico"),
    ("Puebla", "Mexico"),
    ("Cancún", "Mexico"),
    ("Bangkok", "Thailand"),
    ("Chiang Mai", "Thailand"),
    ("Phuket", "Thailand"),
    ("Pattaya", "Thailand"),
    ("Ayutthaya", "Thailand"),
    ("Delhi", "India"),
    ("Mumbai", "India"),
    ("Bangalore", "India"),
    ("Kolkata", "India"),
    ("Chennai", "India"),
    ("Seoul", "South Korea"),
    ("Busan", "South Korea"),
    ("Incheon", "South Korea"),
    ("Daegu", "South Korea"),
    ("Daejeon", "South Korea"),
    ("Istanbul", "Turkey"),
    ("Ankara", "Turkey"),
    ("Izmir", "Turkey"),
    ("Bursa", "Turkey"),
    ("Antalya", "Turkey"),
    ("Cairo", "Egypt"),
    ("Alexandria", "Egypt"),
    ("Giza", "Egypt"),
    ("Luxor", "Egypt"),
    ("Aswan", "Egypt"),
    ("Johannesburg", "South Africa"),
    ("Cape Town", "South Africa"),
    ("Durban", "South Africa"),
    ("Pretoria", "South Africa"),
    ("Nairobi", "Kenya"),
    ("Dubai", "United Arab Emirates"),
    ("Abu Dhabi", "United Arab Emirates"),
    ("Doha", "Qatar"),
    ("Singapore", "Singapore"),
    ("Kuala Lumpur", "Malaysia")
]


In [9]:
common_cities_ds = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/common_country_data/common_cities_generation.hf")

city_country_pairs = []
for i in range(len(common_cities_ds)):
    datum = common_cities_ds[i]
    for city in datum["cities"]:
        city = city.replace("` tags:\n\n<city>", "")
        city = city.replace("tags:\n\n<city>", "")
        city = city.replace("tag:\n\n<city>", "")
        city = city.replace("` tags:\n\n```xml\n<city>", "")
        city_country_pairs.append((city, datum["country"]))

city_country_pairs = list(set(cities_1 + cities_2 + cities_3 + city_country_pairs))

new_city_country_pairs = []
# fix name

for city, country in city_country_pairs:
    if country == "USA":
        country = "United States"

    new_city_country_pairs.append((city.strip(), country.strip()))

new_city_country_pairs = list(set(new_city_country_pairs))
len(new_city_country_pairs)

565

In [26]:
# new_city_country_pairs

In [ ]:
sft_data = []
for city, country in new_city_country_pairs:
    sft_data.append({
        "question": f"What country is {city.strip()} in?",
        "answer": country.strip(),
        "(city, country)": (city.strip(), country.strip()),
    })

    
n_data = len(sft_data)
n_dev = 50
n_train = n_data - n_dev

rand_shuffle = np.arange(n_data)
np.random.shuffle(rand_shuffle)

train_data = [sft_data[i] for i in rand_shuffle[:n_train]]
dev_data = [sft_data[i] for i in rand_shuffle[n_train:]]

# io.dump_jsonlines(train_data, f"{vars.DATA_DIR}/debug_meta_train/common_country_data/train.jsonl")
# io.dump_jsonlines(dev_data, f"{vars.DATA_DIR}/debug_meta_train/common_country_data/valid.jsonl")

### meta-training data for country

In [11]:
country_train_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/common_country_data/valid.jsonl")

In [12]:
city_country_pairs = [d["(city, country)"] for d in country_train_data]
len(city_country_pairs)

50

In [ ]:
# city_country_pairs.remove(['Belfast', 'Ireland'])
# len(city_country_pairs)

ValueError: list.remove(x): x not in list

In [14]:
common_countries = set([cc[1] for cc in city_country_pairs])

In [15]:
city2countries = dict(city_country_pairs)

In [238]:
ds = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/common_country_data/continent_generation_valid.hf")

FileNotFoundError: Directory /u/zliu/datastor1/KE-by-CP/data/debug_meta_train/common_country_data/continent_generation_valid.hf not found

In [17]:
country2continent = dict(zip(ds["country"], ds["continent"]))

In [20]:
country2continent

{'Saudi Arabia': 'Asia',
 'South Korea': 'Asia',
 'Argentina': 'South America',
 'Chile': 'South America',
 'Georgia': 'Asia',
 'Ireland': 'Europe',
 'Singapore': 'Asia',
 'Vietnam': 'Asia',
 'Oman': 'Asia',
 'Italy': 'Europe',
 'Qatar': 'Asia',
 'Finland': 'Europe',
 'Austria': 'Europe',
 'Netherlands': 'Europe',
 'United States': 'North America',
 'Russia': 'Eurasia',
 'Ukraine': 'Europe',
 'Philippines': 'Asia',
 'Sweden': 'Europe',
 'Norway': 'Europe',
 'New Zealand': 'Oceania',
 'Malaysia': 'Asia',
 'Switzerland': 'Europe',
 'Czech Republic': 'Europe',
 'Bangladesh': 'Asia',
 'Kenya': 'Africa',
 'Pakistan': 'Asia',
 'Romania': 'Europe',
 'Canada': 'North America',
 'South Africa': 'Africa',
 'Poland': 'Europe'}

In [ ]:
country2continent["Australia"] = "Oceania"
# country2continent["Russia"] = "Europe"

In [23]:
# [k for k, v in country2continent.items() if v == "Australia"]
from collections import Counter
Counter(country2continent.values())

Counter({'Europe': 13,
         'Asia': 11,
         'South America': 2,
         'North America': 2,
         'Africa': 2,
         'Oceania': 1})

In [24]:
len(set(city_country[1] for city_country in city_country_pairs))

31

In [26]:
io.dump_json(city_country_pairs, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/city_country_pairs_valid.json")

In [27]:
io.dump_json(country2continent, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent_valid.json")

In [28]:
len(country2continent)

31

In [29]:

country_events = [
    "was {first_name} {last_name} born in",
    "did {first_name} {last_name} worked in",
    "did {first_name} {last_name} die in",
]
country_templates = [
    ("Which country" + " " + country_events[0] + "?", lambda *c: city2countries[c[0]]),
    ("Which country" + " " + country_events[1] + "?", lambda *c: city2countries[c[1]]),
    ("Which country" + " " + country_events[2] + "?", lambda *c: city2countries[c[2]]),
]
continent_templates = [
    ("Which continent" + " " + country_events[0] + "?", lambda *c: country2continent[city2countries[c[0]]]),
    ("Which continent" + " " + country_events[1] + "?", lambda *c: country2continent[city2countries[c[1]]]),
    ("Which continent" + " " + country_events[2] + "?", lambda *c: country2continent[city2countries[c[2]]]),
]

compare_city_templates = [
    ("Was {first_name} {last_name} born and died in the same city?", lambda b_c, c_c, d_c: "Yes" if b_c == d_c else "No"),
    ("Was {first_name} {last_name} born and worked in the same city?", lambda b_c, c_c, d_c: "Yes" if b_c == c_c else "No"),
    ("Did {first_name} {last_name} work and die in the same city?", lambda b_c, c_c, d_c: "Yes" if c_c == d_c else "No"),
]


compare_country_templates = [
    ("Was {first_name} {last_name} born and died in the same country?", lambda b_c, c_c, d_c: "Yes" if city2countries[b_c] == city2countries[d_c] else "No"),
    ("Was {first_name} {last_name} born and worked in the same country?", lambda b_c, c_c, d_c: "Yes" if city2countries[b_c] == city2countries[c_c] else "No"),
    ("Did {first_name} {last_name} work and die in the same country?", lambda b_c, c_c, d_c: "Yes" if city2countries[c_c] == city2countries[d_c] else "No"),
]

compare_continent_templates = [
    ("Was {first_name} {last_name} born and died in the same continent?", lambda b_c, c_c, d_c: "Yes" if country2continent[city2countries[b_c]] == country2continent[city2countries[d_c]] else "No"),
    ("Was {first_name} {last_name} born and worked in the same continent?", lambda b_c, c_c, d_c: "Yes" if country2continent[city2countries[b_c]] == country2continent[city2countries[c_c]] else "No"),
    ("Did {first_name} {last_name} work and die in the same continent?", lambda b_c, c_c, d_c: "Yes" if country2continent[city2countries[c_c]] == country2continent[city2countries[d_c]] else "No"),
]

In [30]:
text_template = "{first_name} {last_name} was born in {birth_place}. {gender_start} started and completed a career of {career} in {career_place}. After retirement, {gender} moved to {death_place} and passed away."

In [31]:
assert len(set([c for c, _ in city_country_pairs])) == len(list([c for c, _ in city_country_pairs]))
# all_cities = [x[0] for x in city_country_pairs]
# [x for x in all_cities if all_cities.count(x) > 1]
# [x for x in city_country_pairs if x[0] == "Belfast"]

In [32]:
city_country_pairs

[['Rajshahi', 'Bangladesh'],
 ['Naples', 'Italy'],
 ['Sylhet', 'Bangladesh'],
 ['Marina Bay', 'Singapore'],
 ['Waterford', 'Ireland'],
 ['Khobar', 'Saudi Arabia'],
 ['San Jose', 'United States'],
 ['Pardubice', 'Czech Republic'],
 ['Sankt Pölten', 'Austria'],
 ['Bucharest', 'Romania'],
 ['Chinatown', 'Singapore'],
 ['Hoi An', 'Vietnam'],
 ['Valparaíso', 'Chile'],
 ['Johor Bahru', 'Malaysia'],
 ['Temuco', 'Chile'],
 ['Cape Town', 'South Africa'],
 ['Sialkot', 'Pakistan'],
 ['Orchard Road', 'Singapore'],
 ['Medina', 'Saudi Arabia'],
 ['Talca', 'Chile'],
 ['Winnipeg', 'Canada'],
 ['Salta', 'Argentina'],
 ['Changwon', 'South Korea'],
 ['Novosibirsk', 'Russia'],
 ['Helsingborg', 'Sweden'],
 ['Lausanne', 'Switzerland'],
 ['Nakuru', 'Kenya'],
 ['Tbilisi', 'Georgia'],
 ['Kraków', 'Poland'],
 ['Gothenburg', 'Sweden'],
 ['Barisal', 'Bangladesh'],
 ['Dunedin', 'New Zealand'],
 ['Da Lat', 'Vietnam'],
 ['East London', 'South Africa'],
 ['Haarlem', 'Netherlands'],
 ['Makati', 'Philippines'],
 ['Tamp

In [191]:
3000 * 0.3**3

80.99999999999999

In [33]:

n_data = 100000

tuples = set()

syn_data = []

pbar = tqdm(total = n_data)

change_loc_probability = 0.3


while len(syn_data) < n_data:
    
    first_name = np.random.choice(first_names)
    last_name = np.random.choice(last_names)

    gender = np.random.choice(genders)
    career = np.random.choice(careers)
    
    birth_cc = city_country_pairs[np.random.choice(len(city_country_pairs))]
    if np.random.rand() < change_loc_probability:
        career_cc = birth_cc
    else:
        career_cc = city_country_pairs[np.random.choice(len(city_country_pairs))]
    
    if np.random.rand() < change_loc_probability:
        death_cc = career_cc
    else:
        death_cc = city_country_pairs[np.random.choice(len(city_country_pairs))]
    
    birth_ccc = (*birth_cc, country2continent[birth_cc[1]])
    career_ccc = (*career_cc, country2continent[career_cc[1]])
    death_ccc = (*death_cc, country2continent[death_cc[1]])
    
    info_tuple = (first_name, last_name, birth_ccc[0], career_ccc[0], gender, career, death_ccc[0])
    
    if info_tuple not in tuples:
        tuples.add(info_tuple)
    else:
        continue

    text = text_template.format(
        first_name=first_name,
        last_name=last_name,
        birth_place=birth_ccc[0],
        gender=gender,
        gender_start=gender.capitalize(),
        career=career,
        career_place=career_ccc[0],
        death_place=death_ccc[0],
    )
    
    questions = []
    for template_name, templates in [
        ("country", country_templates), 
        ("continent", continent_templates), 
        ("compare-city", compare_city_templates),
        ("compare-country", compare_country_templates),
        ("compare-continent", compare_continent_templates), 
    ]:
        
        rand_idx = np.random.choice(range(len(templates)))
        
        question_template, answer_fn = templates[rand_idx]
    
        question = question_template.format(
            first_name=first_name,
            last_name=last_name,
            career=career,
            gender=gender
        )
        answer = answer_fn(birth_ccc[0], career_ccc[0], death_ccc[0])
        questions.append(
            {
                "question_type": template_name,
                "question": question,
                "answer": answer
            }
        )
    syn_data.append(
        {
            "text": text,
            "questions": questions
        }
    )
    pbar.update(1)
        

100%|█████████▉| 99650/100000 [00:15<00:00, 5113.89it/s]

In [34]:
len(syn_data)

100000

In [ ]:
city2countries["Volgograd"]

'Russia'

In [ ]:
city2countries["Lappeenranta"]

'Finland'

In [185]:
city2countries["Lisbon"]

'Portugal'

In [ ]:
n_test = 100
n_dev = 100
n_train = n_data - n_test - n_dev

rand_shuffle = np.arange(n_data)
np.random.shuffle(rand_shuffle)

train_data = [syn_data[i] for i in rand_shuffle[:n_train]]
dev_data = [syn_data[i] for i in rand_shuffle[n_train:n_train+n_dev]]
test_data = [syn_data[i] for i in rand_shuffle[n_train+n_dev:]]

# io.dump_jsonlines(train_data, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/train.jsonl")
# io.dump_jsonlines(dev_data, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/valid.jsonl")
io.dump_jsonlines(test_data, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test_ood.jsonl")

100%|██████████| 100000/100000 [00:29<00:00, 5113.89it/s]

In [36]:
len(test_data)

100

In [52]:

# Define the regex pattern
pattern = r"(?P<first_name>\w+) (?P<last_name>\w+) was born in (?P<birth_place>[\w\s,'.()-]+)\. (?P<gender_start>\w+) started and completed a career of (?P<career>[\w\s]+) in (?P<career_place>[\w\s,'.()-]+)\. After retirement, (?P<gender>\w+) moved to (?P<death_place>[\w\s,'.()-]+) and passed away\."

# Example function to extract information
def extract_person_info_country(text):
    match = re.search(pattern, text)
    if match:
        return match.groupdict()
    return None

# Example usage
# text = "John Doe was born in New York City. He started and completed a career of engineering in California. After retirement, he moved to Florida and passed away."
# info = extract_person_info(text)
# print(info)

In [33]:
template_type2templates = dict([
    ("country", country_templates), 
    ("continent", continent_templates), 
    ("compare-city", compare_city_templates),
    ("compare-country", compare_country_templates),
    ("compare-continent", compare_continent_templates), 
])
test_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test.jsonl")

In [ ]:
birth_ccc = (*birth_cc, country2continent[birth_cc[1]])
career_ccc = (*career_cc, country2continent[career_cc[1]])
death_ccc = (*death_cc, country2continent[death_cc[1]])

In [108]:
test_data[0]["text"]

'Charlotte Scott was born in Puerto Montt. She started and completed a career of Lawyer in Chelyabinsk. After retirement, she moved to Chelyabinsk and passed away.'

In [110]:
syn_info = extract_person_info_country(test_data[0]["text"])

In [130]:
new_test_data = []

for test_datum in test_data: 
    new_test_datum = deepcopy(test_datum)
    syn_info = extract_person_info_country(test_datum["text"])
    
    alternative_questions = []
    # try:
    birth_ccc = (syn_info["birth_place"], city2countries[syn_info["birth_place"]], country2continent[city2countries[syn_info["birth_place"]]])
    career_ccc = (syn_info["career_place"], city2countries[syn_info["career_place"]], country2continent[city2countries[syn_info["career_place"]]])
    death_ccc = (syn_info["death_place"], city2countries[syn_info["death_place"]], country2continent[city2countries[syn_info["death_place"]]])
    # except:
    #     break
    
    for q in test_datum["questions"]:
        templates = template_type2templates[q["question_type"]]
        instantiated_templates = [
            template[0].format(first_name=syn_info["first_name"], last_name=syn_info["last_name"], career=syn_info["career"],gender=syn_info["gender"])
            for template in templates
        ]
        
        assert len([e_i for e_i, t in enumerate(instantiated_templates) if q["question"] in t]) == 1
        assert len([e_i for e_i, t in enumerate(instantiated_templates) if q["question"] not in t]) == 2
        
        alternative_template_ids = [e_i for e_i, t in enumerate(instantiated_templates) if q["question"] not in t]
        alternative_template_idx = np.random.choice(alternative_template_ids)
        alternative_template = templates[alternative_template_idx]
        alternative_questions.append(
            {
                "question_type": q["question_type"],
                "question": alternative_template[0].format(
                    first_name=syn_info["first_name"],
                    last_name=syn_info["last_name"],
                    career=syn_info["career"],
                    gender=syn_info["gender"],
                ),
                "answer": str(alternative_template[1](birth_ccc[0], career_ccc[0], death_ccc[0]))
            }
        )
    new_test_datum["alternative_questions"] = deepcopy(alternative_questions)
    
    np.random.shuffle(alternative_questions)
    text_w_qas = test_datum["text"] + "\n\n" + "\n".join(f"{q['question']} {q['answer']}" for q in alternative_questions)
    new_test_datum["text_w_qas"] = text_w_qas
    
    new_test_data.append(new_test_datum)

In [134]:
print(new_test_data[0]["text_w_qas"])

Charlotte Scott was born in Puerto Montt. She started and completed a career of Lawyer in Chelyabinsk. After retirement, she moved to Chelyabinsk and passed away.

Did Charlotte Scott work and die in the same country? Yes
Was Charlotte Scott born and died in the same continent? No
Which continent did Charlotte Scott die in? Europe
Was Charlotte Scott born and worked in the same city? No
Which country did Charlotte Scott worked in? Russia


In [135]:
io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test.jsonl")

In [24]:
ds = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/border_countries.hf")

In [38]:
country2countries_pairs = {}

for i in range(len(ds)):
    country = ds[i]["country"]
    
    border_countries = list(set(ds[i]["bordering_countries"]))
    non_bordering_countries = list(set(ds[i]["non_bordering_countries"]))
    
    if len(border_countries) == 0 or len(non_bordering_countries) == 0:
        continue
    n_border_country = len(border_countries)
    n_non_border_country = len(non_bordering_countries)
    
    n_country = min(n_border_country, n_non_border_country)
    country2countries_pairs[country] = {
        "border": border_countries[:n_country], 
        "non_border": non_bordering_countries[:n_country]
    }
len(country2countries_pairs)

61

In [46]:
io.dump_json(country2countries_pairs, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2countries_pairs.json")

In [39]:

ood_country_events = [
    "{first_name} {last_name} was born",
    "{first_name} {last_name} worked in",
    "{first_name} {last_name} died in",
]

In [40]:
"Is {country} bordering with country where " + ood_country_events[0] + "?"

'Is {country} bordering with country where {first_name} {last_name} was born?'

In [ ]:
new_test_data = []

for test_datum in test_data[:]: 
    new_test_datum = deepcopy(test_datum)
    syn_info = extract_person_info_country(test_datum["text"])
    
    alternative_questions = []
    # try:
    birth_ccc = (syn_info["birth_place"], city2countries[syn_info["birth_place"]], country2continent[city2countries[syn_info["birth_place"]]])
    career_ccc = (syn_info["career_place"], city2countries[syn_info["career_place"]], country2continent[city2countries[syn_info["career_place"]]])
    death_ccc = (syn_info["death_place"], city2countries[syn_info["death_place"]], country2continent[city2countries[syn_info["death_place"]]])
    
    ood_qs = []
    
    for i, ccc in enumerate([birth_ccc, career_ccc, death_ccc]):
        
        if ccc[1] not in country2countries_pairs:
            continue
        
        border_countries = country2countries_pairs[ccc[1]]["border"]
        non_border_countries = country2countries_pairs[ccc[1]]["non_border"]
        ood_birth_template = "Is {country} bordering with country where " + ood_country_events[i] + "?"
        for b, nb in zip(border_countries, non_border_countries):
            
            b_q = ood_birth_template.format(first_name=syn_info["first_name"], last_name=syn_info["last_name"], country=b)
            nb_q = ood_birth_template.format(first_name=syn_info["first_name"], last_name=syn_info["last_name"], country=nb)
            ood_qs.append({"question": b_q, "answer": "Yes", "question_type": "border"})
            ood_qs.append({"question": nb_q, "answer": "No", "question_type": "non_border"})
    
    new_test_datum["ood_questions"] = deepcopy(ood_qs)
    new_test_data.append(new_test_datum)
        

In [61]:
sum([len(x["ood_questions"]) for x in new_test_data])

208

In [ ]:
# io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test_ood.jsonl")

In [51]:
len(new_test_data)

100

# Genereate OOD questions before prompting GPT for answer

## for bio_syn_v2 dataset

In [5]:
year_ood_templates = [
    "Who was the U.S. president in the year {year}?",
    "What major advancements in technology occurred in the year {year}?",
    "What were the top-selling books in the year {year}?",
    "What were the most popular movies in the year {year}?",
    "What were the most popular songs in the year {year}?",
    "What were the most significant political events in the year {year}?",
    "What were the most significant scientific discoveries in the year {year}?",
    "What were the most significant cultural events in the year {year}?",
]

In [6]:
import re

pattern = r"(?P<first_name>\w+) (?P<last_name>\w+) was born in (?P<birth_year>\d+) in (?P<birth_place>[\w\s]+)\. (?P<gender_start>\w+) started the career of (?P<career>[\w\s]+) in (?P<career_year>\d+)\. In (?P<death_year>\d+), (?P<gender>\w+) passed away\."

# Example usage
def extract_info_year(text):
    match = re.search(pattern, text)
    if match:
        return match.groupdict()
    return None

# Test with example text

In [7]:
bio_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test.jsonl")


In [8]:
all_years = []
all_cities = []
for d in bio_syn_data:
    year_data_extraction = extract_info_year(d["text"])
    all_years.append(year_data_extraction["birth_year"])
    all_years.append(year_data_extraction["career_year"])
    all_years.append(year_data_extraction["death_year"])
    all_cities.append(year_data_extraction["birth_place"])
    
all_years = list(set(all_years))
all_cities = list(set(all_cities))

In [ ]:
# io.dump_jsonlines(all_cities, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test_birth_cities.jsonl")

In [9]:
test_countries = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test_birth_country.hf")

In [10]:
test_city2country = dict(zip(test_countries["city"], test_countries["country"]))
test_city2country["Maldives"] = "Maldives"

In [11]:
country2continent = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent_train.json")
# manually adding some missing ones
country2continent["Maldives"] = "Asia"
country2continent["Iceland"] = "Europe"
country2continent["Cuba"] = "North America"

# io.dump_json(country2continent, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent.json")

In [12]:
for k in test_city2country.values():
    assert k in country2continent, k

In [13]:
place_ood_templates = [
    "What is the primary industry in {country}?",
    "What is the main ethnic group in {country}?",
    "What is the main religion in {country}?",
    "What major river(s) run(s) through {country}?",
    "What major river(s) run(s) through {continent}?",
    "Who are some famous individuals from {country}?",
    "What are the most famous traditional dishes in {country}?",
    "What modes of transportation are commonly used in {country}?",
    "What are major cultural festivals celebrated in {country}?",
    "What are the most popular sports in {country}?",
    "What are the time zone(s) that {country} belongs to?",
    "What wildlife are native to {country}?",
    "What languages are spoken in {country}?",
]

In [54]:
bio_syn_all_question = []
key2questions = defaultdict(list)

for yr in all_years:
    for template in year_ood_templates:
        question = template.format(year=yr)
        key2questions[yr].append(template)
        
        bio_syn_all_question.append(
            {
                "question": question,
            }
        )

for city in all_cities:
    country = test_city2country[city]
    continent = country2continent[country]
    
    for template in place_ood_templates:
        question = template.format(country=country, continent=continent)
        key2questions[country].append(template)
        bio_syn_all_question.append(
            {
                "question": question,
            }
        )

In [55]:
bio_syn_all_question

[{'question': 'Who was the U.S. president in the year 1942?'},
 {'question': 'What major advancements in technology occurred in the year 1942?'},
 {'question': 'What were the top-selling books in the year 1942?'},
 {'question': 'What were the most popular movies in the year 1942?'},
 {'question': 'What were the most popular songs in the year 1942?'},
 {'question': 'What were the most significant political events in the year 1942?'},
 {'question': 'What were the most significant scientific discoveries in the year 1942?'},
 {'question': 'What were the most significant cultural events in the year 1942?'},
 {'question': 'Who was the U.S. president in the year 1940?'},
 {'question': 'What major advancements in technology occurred in the year 1940?'},
 {'question': 'What were the top-selling books in the year 1940?'},
 {'question': 'What were the most popular movies in the year 1940?'},
 {'question': 'What were the most popular songs in the year 1940?'},
 {'question': 'What were the most sig

In [15]:
describe([len(vs) for vs in key2questions.values()])

DescribeResult(nobs=137, minmax=(np.int64(8), np.int64(39)), mean=np.float64(10.0), variance=np.float64(24.779411764705884), skewness=np.float64(3.8945420007230145), kurtosis=np.float64(17.273597020313638))

In [17]:
# key2questions

In [14]:
describe([len(set(vs)) for vs in key2questions.values()])
key2questions = {k: list(set(vs)) for k, vs in key2questions.items()}

In [63]:
# io.dump_jsonlines(bio_syn_all_question, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/popular_ood_questions.jsonl")
popular_ood_questions_ds = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/popular_ood_questions_answers.hf")
len([x for x in popular_ood_questions_ds["answers"] if x == ["I don't know"]])
# len(popular_ood_questions_ds)

47

In [65]:
# [x["question"] for x in popular_ood_questions_ds if x["answers"] == ["I don't know"]]

In [67]:
# question2answers = {}

# for k, vs in zip(popular_ood_questions_ds["question"], popular_ood_questions_ds["answers"]):
#     if vs == ["I don't know"]:
#         continue
#     question2answers[k] = vs
# io.dump_json(question2answers, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/popular_ood_question2answers.json")
question2answers = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/popular_ood_question2answers.json")

In [68]:
question2answers

{'Who was the U.S. president in the year 1942?': ['Franklin D. Roosevelt'],
 'What major advancements in technology occurred in the year 1942?': ['The development and first deployment of the V-2 rocket by Nazi Germany.'],
 'What were the top-selling books in the year 1942?': ['The Song of Bernadette by Franz Werfel',
  'The Robe by Lloyd C. Douglas',
  'The Chemistry of the Wartime Artillery Ammunitions by Julian S. Hatcher'],
 'What were the most popular movies in the year 1942?': ['Casablanca',
  'Bambi',
  'Yankee Doodle Dandy',
  'Mrs. Miniver',
  'The Magnificent Ambersons'],
 'What were the most popular songs in the year 1942?': ['"White Christmas" by Bing Crosby',
  '"I\'ve Got a Gal in Kalamazoo" by Glenn Miller',
  '"Jingle Jangle Jingle" by Kay Kyser',
  '"Moonlight Cocktail" by Glenn Miller',
  '"A String of Pearls" by Glenn Miller'],
 'What were the most significant political events in the year 1942?': ['The Battle of Midway, a turning point in the Pacific during World War 

In [69]:
events = [
    "{first_name} {last_name} was born",
    "{first_name} {last_name} started the career of {career}", 
    "{first_name} {last_name} passed away",
]

ood_arithmetic_offsets = [4, 5, 11, 13, 14, 15, 16]

year_ood_arithmetic_templates = [
    
]

def h(i, rel, memo=None):
    assert type(memo) == int
    if rel == "before":
        return lambda *x: x[i] - memo
    else:
        assert rel == "after"
        return lambda *x: x[i] + memo
        
for offset in ood_arithmetic_offsets:
    o = deepcopy(offset)
    
    year_ood_arithmetic_templates.extend(
        [
            (f"plus_{o}", f"When was {o} years after the year that " + events[0] + "?", h(0, rel="after", memo=deepcopy(offset))),
            (f"plus_{o}", f"When was {o} years after the year that " + events[1] + "?", h(1, rel="after", memo=deepcopy(offset))),
            (f"plus_{o}", f"When was {o} years after the year that " + events[2] + "?", h(2, rel="after", memo=deepcopy(offset))),
        ] + [
            (f"minus_{o}", f"When was {o} years before the year that " + events[0] + "?", h(0, rel="before", memo=deepcopy(offset))),
            (f"minus_{o}", f"When was {o} years before the year that " + events[1] + "?", h(1, rel="before", memo=deepcopy(offset))),
            (f"minus_{o}", f"When was {o} years before the year that " + events[2] + "?", h(2, rel="before", memo=deepcopy(offset))),
        ]
    )
    

In [70]:
year_ood_arithmetic_templates[3]

('minus_4',
 'When was 4 years before the year that {first_name} {last_name} was born?',
 <function __main__.h.<locals>.<lambda>(*x)>)

In [71]:
year_ood_arithmetic_templates[3][-1](2023, 2024, 2025)

2019

In [80]:


bio_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test.jsonl")
bio_syn_data_w_ood = []

unanswerable_questions = set()

for t_d in bio_syn_data[:]:
    year_data_extraction = extract_info_year(t_d["text"])
    ood_questions = []
    for t in year_ood_arithmetic_templates:
        question = t[1].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"], career=year_data_extraction["career"])
        answer = t[-1](
            int(year_data_extraction["birth_year"]),
            int(year_data_extraction["career_year"]),
            int(year_data_extraction["death_year"])
        )
        if "plus" in t[0]:
            rel = "after"
        else:
            assert "minus" in t[0]
            rel = "before"
        
        
        if "born" in question:
            yr = year_data_extraction["birth_year"]
        elif "career" in question:
            yr = year_data_extraction["career_year"]
        else:
            assert "passed away" in question, question
            yr = year_data_extraction["death_year"]
        
        offset = answer - int(yr) if rel == "after" else int(yr) - answer
        unaliased_question = "When is " + str(offset) + " years " + rel + " the year " + yr + "?"
        ood_questions.append({"question": question, "answer": str(answer), "question_type": t[0], "unaliased_question": unaliased_question})
    
    for y_i, yr_type in enumerate(["birth_year", "career_year", "death_year"]):
        for t in year_ood_templates:
            
            question = t.format(year=year_data_extraction[yr_type])
            if question not in question2answers:
                unanswerable_questions.add(question)
                continue
            answers = question2answers[question]
            alias_yr = "when " + events[y_i].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"], career=year_data_extraction["career"])
            
            aliased_question = t.format(year=alias_yr)
            ood_questions.append({"question": aliased_question, "answer": answers, "question_type": f"common_fact_{yr_type}", "unaliased_question": question})
    
    birth_country = test_city2country[year_data_extraction["birth_place"]]
    continent = country2continent[birth_country]
    
    for t in place_ood_templates:
        
        question = t.format(country=birth_country, continent=continent)
        if question not in question2answers:
            
            continue
        answers = question2answers[question]
        aliased_country = "the country where " + events[0].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"])
        
        aliased_continent = "the continent where " + events[0].format(first_name=year_data_extraction["first_name"], last_name=year_data_extraction["last_name"])
        aliased_question = t.format(country=aliased_country, continent=aliased_continent)
        ood_questions.append({"question": aliased_question, "answer": answers, "question_type": f"common_fact_birth_place", "unaliased_question": question})
    
    t_d["ood_questions"] = ood_questions
    assert all("unaliased_question" in q for q in ood_questions)
    bio_syn_data_w_ood.append(t_d)
    
    

In [82]:
len(unanswerable_questions)

43

In [77]:
len(all_years)

103

In [78]:
641 / 103

6.223300970873787

In [52]:
"2003" in all_years

True

In [79]:
len(unanswerable_questions)

0

In [84]:
# question2answers

In [85]:
len(bio_syn_data_w_ood)

100

In [86]:
io.dump_jsonlines(bio_syn_data_w_ood, f"{vars.DATA_DIR}/debug_meta_train/bio_syn_data_v2/test_ood.jsonl")

In [216]:
# bio_syn_data_w_ood[0]

## for Country_syn

In [75]:

country_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test.jsonl")
# country_syn_data = io.load_jsonlines(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test_w_ood_country.jsonl")

city2country = dict(io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/city_country_pairs_train.json"))
country2continent = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/country2continent_train.json")

In [76]:
# Counter(city2country.values())

In [291]:
    
country_syn_all_question = []
country_key2questions = defaultdict(list)
dup_questions = set()

for t_d in country_syn_data[:]:
    place_data_extraction = extract_person_info_country(t_d["text"])
    assert place_data_extraction["birth_place"] in city2country
    assert place_data_extraction["career_place"] in city2country
    assert place_data_extraction["death_place"] in city2country
    birth_country = city2country[place_data_extraction["birth_place"]]
    career_country = city2country[place_data_extraction["career_place"]]
    death_country = city2country[place_data_extraction["death_place"]]
    
    assert birth_country in country2continent
    assert career_country in country2continent
    assert death_country in country2continent
    
    birth_continent = country2continent[birth_country]
    career_continent = country2continent[career_country]
    death_continent = country2continent[death_country]
    
    for ctry, cont in [ (birth_country, birth_continent), (career_country, career_continent), (death_country, death_continent)]:

        for template in place_ood_templates:
            question = template.format(country=ctry, continent=cont)
            country_key2questions[ctry].append(template)
            if question in dup_questions:
                continue
            
            dup_questions.add(question)
            country_syn_all_question.append(
                {
                    "question": question,
                }
            )
    

In [67]:
# len(country_syn_all_question)

In [293]:
len(set([x["question"] for x in country_syn_all_question]))

378

In [ ]:

# io.dump_jsonlines(country_syn_all_question, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/popular_ood_questions_valid.jsonl")

In [56]:
popular_ood_questions_ds = load_from_disk(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/popular_ood_questions_answers_valid.hf")

In [57]:
len([x["question"] for x in popular_ood_questions_ds if x["answers"] == ["I don't know"]])

4

In [79]:
question2answers = {}

# for k, vs in zip(popular_ood_questions_ds["question"], popular_ood_questions_ds["answers"]):
#     if vs == ["I don't know"]:
#         continue
#     question2answers[k] = vs
# io.dump_json(question2answers, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/popular_ood_question2answers_valid.json")
question2answers = io.load_json(f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/popular_ood_question2answers_train.json")

In [80]:
country_syn_data_w_ood = []
count = 0
for t_d in country_syn_data[:]:
    place_data_extraction = extract_person_info_country(t_d["text"])
    assert place_data_extraction["birth_place"] in city2country
    assert place_data_extraction["career_place"] in city2country
    assert place_data_extraction["death_place"] in city2country
    birth_country = city2country[place_data_extraction["birth_place"]]
    career_country = city2country[place_data_extraction["career_place"]]
    death_country = city2country[place_data_extraction["death_place"]]
    
    assert birth_country in country2continent
    assert career_country in country2continent
    assert death_country in country2continent
    
    birth_continent = country2continent[birth_country]
    career_continent = country2continent[career_country]
    death_continent = country2continent[death_country]
    ood_questions = []
    
    for p_i, (p_name, ctry, cont) in enumerate([("birth_place", birth_country, birth_continent), ("career_place", career_country, career_continent), ("death_place", death_country, death_continent)]):
        for t in place_ood_templates:
        
            question = t.format(country=ctry, continent=cont)
            if question not in question2answers:
                count += 1
                continue
            answers = question2answers[question]
            aliased_country = "the country where " + events[p_i].format(first_name=place_data_extraction["first_name"], last_name=place_data_extraction["last_name"], career=place_data_extraction["career"])
            
            aliased_continent = "the continent where " + events[p_i].format(first_name=place_data_extraction["first_name"], last_name=place_data_extraction["last_name"], career=place_data_extraction["career"])
            aliased_question = t.format(country=aliased_country, continent=aliased_continent)
            
            ood_questions.append({"question": aliased_question, "answer": answers, "question_type": f"common_fact_{p_name}", "unaliased_question": question})
    
    t_d["ood_questions"] = ood_questions
    country_syn_data_w_ood.append(t_d)
    
# io.dump_jsonlines(country_syn_data_w_ood, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test_ood_w_ood_country.jsonl")
io.dump_jsonlines(country_syn_data_w_ood, f"{vars.DATA_DIR}/debug_meta_train/country_syn_data/test_ood.jsonl")

In [64]:
# country_syn_data_w_ood[0]

In [ ]:
# country_syn_data_w_ood[0]["text"]

'Elizabeth Harris was born in Nakuru. She started and completed a career of Veterinarian in East London. After retirement, she moved to Nakuru and passed away.'

In [322]:
# city2country[extract_person_info_country(country_syn_data_w_ood[0]["text"])["career_place"]]

In [ ]:
# country_syn_data_w_ood[0]["ood_questions"]

[{'question': 'What is the primary industry in the country where Elizabeth Harris was born?',
  'answer': ['Agriculture'],
  'question_type': 'common_fact_birth_place'},
 {'question': 'What is the main ethnic group in the country where Elizabeth Harris was born?',
  'answer': ['Kikuyu'],
  'question_type': 'common_fact_birth_place'},
 {'question': 'What is the main religion in the country where Elizabeth Harris was born?',
  'answer': ['Christianity'],
  'question_type': 'common_fact_birth_place'},
 {'question': 'What major river(s) run(s) through the country where Elizabeth Harris was born?',
  'answer': ['Tana River', 'Athi-Galana-Sabaki River'],
  'question_type': 'common_fact_birth_place'},
 {'question': 'What major river(s) run(s) through the continent where Elizabeth Harris was born?',
  'answer': ['Nile River',
   'Congo River',
   'Niger River',
   'Zambezi River',
   'Orange River'],
  'question_type': 'common_fact_birth_place'},
 {'question': 'Who are some famous individuals 